<a href="https://colab.research.google.com/github/salarbalou/steadystate_analysis/blob/main/plot_photothermal_Tvst_profile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import requests as req
import plotly.io as pio
import re
import string as sr
pio.templates.default ='plotly_dark'
from plotly.subplots import make_subplots
import plotly.express as px

In [2]:
def findmb(x, y):
  m,b = np.polyfit(x, y, deg= 1)
  return m,b

In [66]:
#date_hint = '10-19-21'
#compound_hint = 'GNR , GNS , CQD'
#concentration_hint = '10x, 5x, 1.5x'
#duty_cycle_hint = 'dc20' , 'dc30'
dcs= ['dc20', 'dc30', 'dc40', 'dc50', 'dc60', 'dc70', 'dc80', 'dc90']
all_equ_temp = []
all_laser_power_density = []
for dc in dcs:
 baseurl = 'https://raw.githubusercontent.com/salarbalou/steadystate_analysis/main/' 
 char = '.txt'
 compound_and_concentration_dutycycle= '10-21-21-CQD-10x-'+dc
 text = req.get(baseurl+compound_and_concentration_dutycycle+char).text
 data_path = re.compile(r'^([0-9.]+)[\s,]+([0-9.]+)+[\s,]+([-0-9.]+)+[\s,]+([0-9]+)', re.MULTILINE)
 data_list = data_path.findall(text)
 data = []
 for quadruple in data_list:
      eachquadruple = []
      for abcd in quadruple:
           eachquadruple.append(float(abcd))
      data.append((eachquadruple[0], eachquadruple[1], eachquadruple[2], eachquadruple[3]))
 final_data = np.array(data)
 time = final_data[:, 0][1:162]
 temperature = final_data[:, 1][1:162]
 temperature1 = temperature[22:]
 equ_temp = np.max(temperature, axis=0)
 all_equ_temp.append(equ_temp)
 current = np.sum(-final_data[:, 2][24:162], axis = 0)/138 #Amps
 dutycycle = final_data[:, 3][23:162]
 beam_height = 0.1 #cm
 beam_width = 0.1 #cm
 voltage_value = np.linspace(6, 7.5, num = 250)
 dutycycle_value  = np.arange(0, 250,1)
 m, b = findmb(dutycycle_value, voltage_value) 
 realvoltage = np.sum(m*dutycycle+b)/dutycycle.size #V
 laser_power_density = np.array(((realvoltage*current)/(beam_height*beam_width))) #W/cm^2
 all_laser_power_density.append(laser_power_density)
 dt = np.gradient(time)
 dT = np.gradient(temperature)
 diffTt = np.sqrt((dT/dt)**2)
 fig = make_subplots(rows = 1, cols = 2, subplot_titles=('Temperature vs time at '+dc, 'dT/dt vs time at '+dc))
 fig.add_scatter(x = time, y = temperature, mode = 'markers+lines', marker_color = 'red', row = 1, col = 1, name = 'Temperature vs time')
 fig.add_scatter(x = time, y = diffTt, mode = 'markers+lines', marker_color ='red' , line_color = "green", row  = 1, col  =2, name='dT/dt vs time')
 fig.update_xaxes(title_text = 'time (seconds)', row = 1, col = 1)
 fig.update_xaxes(title_text = 'time (seconds)', row = 1, col = 2)
 fig.update_yaxes(title_text = 'Temperature', row = 1, col = 1)
 fig.update_yaxes(title_text = 'dT/dt', row = 1, col = 2)
 fig.update_layout(height = 800, width = 2400)
if (np.array(all_equ_temp).size>1):
 fig2 = make_subplots()
 fig2.add_scatter(x = all_laser_power_density, y= all_equ_temp, mode = 'markers+lines', name = 'Temperature vs Laser Power Density', marker_color = 'red')
 fig2.update_layout(height = 800, width= 2400)
 fig2.update_xaxes(title = "Laser Power Density (W/cm^2)")
 fig2.update_yaxes(title = 'Temperature')
else:
 print('No Results')

In [67]:
m,b=np.polyfit(np.arange(20, 100, 10), np.array(all_equ_temp), deg=1)
def givemedc(T):
  req_dc = (T-b)/m
  return np.int(req_dc)

In [68]:
givemedc(65)

45